In [1]:
import os

os.chdir("/mnt/d/데이터분석/Dacon/Lettuce_growing_AI")
project_path = os.getcwd()

project_path

'/mnt/d/데이터분석/Dacon/Lettuce_growing_AI'

In [4]:
import random
import pandas as pd
import numpy as np
import math
import glob

import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore')

## Hyper Parameter Setting

In [5]:
CFG = {
    'EPOCHS' : 100,
    'LEARNING_RATE': 1e-3,
    'BATCH_SIZE': 16,
    'SEED':2358
}

In [6]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything(CFG['SEED'])

In [7]:
all_input_list = sorted(glob.glob('dataset/train_input/*.csv'))
all_target_list = sorted(glob.glob('dataset/train_target/*.csv'))

In [9]:
train_input_list = all_input_list[:25]
train_target_list = all_target_list[:25]

val_input_list = all_input_list[25:]
val_target_list = all_target_list[25:]

print(f'학습: {len(train_input_list)}, 검증:{len(val_input_list)}')

학습: 25, 검증:3


## CustomDataset

In [12]:
class Dataloader(tf.keras.utils.Sequence):
    def __init__(self, input_paths, target_paths, batch_size, infer_mode, shuffle=False):
        self.input_paths = input_paths
        self.target_paths = target_paths
        self.batch_size = batch_size
        self.infer_mode = infer_mode
        self.shuffle = shuffle

        self.data_list = []
        self.label_list = []
        print('Data Pre-processing..')
        for input_path, target_path in tqdm(zip(self.input_paths, self.target_paths)):
            input_df = pd.read_csv(input_path)
            target_df = pd.read_csv(target_path)

            input_df = input_df.drop(columns=['obs_time'])
            input_df = input_df.fillna(0)


            target_length = int(len(target_df))

            for idx in range(target_length):
                time_series = input_df[24*idx:24*(idx+1)].values
                self.data_list.append(time_series)

            for label in target_df['predicted_weight_g']:
                self.label_list.append(label)
        print('Done. \n')
        self.on_epoch_end()

    def __len__(self):
        return math.ceil(len(self.data_list)/self.batch_size)

    def __getitem__(self, idx):
        indices = self.indices[idx*self.batch_size:(idx+1)*self.batch_size]

        data = [self.data_list[i] for i in indices]
        label = [self.label_list[i] for i in indices]

        if self.infer_mode == False:
            return tf.convert_to_tensor(data), tf.convert_to_tensor(label)
        else:
            return tf.convert_to_tensor(data)

    def on_epoch_end(self):
        self.indices = np.arange(len(self.data_list))

        if self.shuffle == True:
            np.random.shuffle(self.indices)

In [13]:
train_loader = Dataloader(train_input_list, train_target_list, CFG['BATCH_SIZE'], False, shuffle=True)
val_loader = Dataloader(val_input_list, val_target_list, CFG['BATCH_SIZE'], False, shuffle=True)

Data Pre-processing..


25it [00:00, 37.19it/s]


Done. 

Data Pre-processing..


3it [00:00, 54.55it/s]

Done. 



## Model Define

In [14]:
class BaseModel(tf.keras.Model):
    def __init__(self):
        super(BaseModel, self).__init__()
        self.lstm = tf.keras.layers.LSTM(256)
        self.classifier = tf.keras.layers.Dense(1)

    def call(self, inputs):
        h = self.lstm(inputs)
        return self.classifier(h)

## Train

In [15]:
model = BaseModel()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=CFG['LEARNING_RATE']),
    loss=tf.keras.losses.MeanAbsoluteError()
)

model.fit(
    train_loader, validation_data=val_loader,
    epochs=CFG['EPOCHS'],
    callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-8, verbose=1)])

Epoch 1/100
44/44 [==============================] - 2s 19ms/step - loss: 34.4708 - val_loss: 20.7735 - lr: 0.0010
Epoch 2/100
44/44 [==============================] - 1s 14ms/step - loss: 32.6110 - val_loss: 19.6794 - lr: 0.0010
Epoch 3/100
44/44 [==============================] - 1s 14ms/step - loss: 31.6620 - val_loss: 19.1235 - lr: 0.0010
Epoch 4/100
44/44 [==============================] - 1s 14ms/step - loss: 31.1956 - val_loss: 18.9961 - lr: 0.0010
Epoch 5/100
44/44 [==============================] - 1s 14ms/step - loss: 30.7154 - val_loss: 18.8460 - lr: 0.0010
Epoch 6/100
44/44 [==============================] - 1s 15ms/step - loss: 30.3798 - val_loss: 18.8525 - lr: 0.0010
Epoch 7/100
44/44 [==============================] - 1s 15ms/step - loss: 30.1452 - val_loss: 18.7819 - lr: 0.0010
Epoch 8/100
44/44 [==============================] - 1s 15ms/step - loss: 29.9910 - val_loss: 18.6940 - lr: 0.0010
Epoch 9/100
44/44 [==============================] - 1s 15ms/step - loss: 29.884

In [16]:
test_input_list = sorted(glob.glob('dataset/test_input/*.csv'))
test_target_list = sorted(glob.glob('dataset/test_target/*.csv'))

In [17]:
for test_input_path, test_target_path in zip(test_input_list, test_target_list):
    print(test_target_path)
    test_loader = Dataloader([test_input_path], [test_target_path], CFG['BATCH_SIZE'], True, shuffle=False)
    model_pred = model.predict(test_loader)

    submit_df = pd.read_csv(test_target_path)
    submit_df['predicted_weight_g'] = model_pred
    submit_df.to_csv(test_target_path, index=False)

dataset/test_target/TEST_01.csv
Data Pre-processing..


1it [00:00, 15.33it/s]

Done. 



2/2 [==============================] - 0s 6ms/step
dataset/test_target/TEST_02.csv
Data Pre-processing..


1it [00:00, 34.69it/s]

...

Done. 



2/2 [==============================] - 0s 5ms/step
dataset/test_target/TEST_05.csv
Data Pre-processing..


1it [00:00, 37.52it/s]

Done. 

1/2 [==============>...............] - ETA: 0s

2/2 [==============================] - 0s 6ms/step


In [23]:
import zipfile
# os.chdir("dataset/test_target/")
submission = zipfile.ZipFile("submission.zip", 'w')
for path in test_target_list:
    path = path.split('/')[-1]
    submission.write(path)
submission.close()

In [24]:
os.getcwd()

'/mnt/d/데이터분석/Dacon/Lettuce_growing_AI/dataset/test_target'